In [1]:
import os
import urllib.request

import torch
import torch.nn as nn
from torch.nn import functional as F


In [2]:
use_cuda = torch.cuda.is_available()
use_mps = torch.backends.mps.is_available()

if use_cuda:
    device = torch.device("cuda")
elif use_mps:
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")


Using device: cuda


In [3]:
# hyperparameters
batch_size = 16  # how many independent sequences will we process in parallel?
block_size = 256  # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 500
learning_rate = 3e-4
eval_iters = 200
n_embd = 384 // 2
head_size = 24
n_value_functions = 4
n_key_functions = 4
n_query_functions = 4
n_layer = 2
dropout = 0.2
functions_expansion_factor = 2
projection_expansion_factor = 2
ffwd_expansion_factor = 2
enable_ffwd_layer = True


In [4]:
torch.manual_seed(1337)


In [5]:
file_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
path = "../datasets/shakespeare_char"
file_path = f"{path}/input.txt"

if not os.path.isfile(file_path):
    os.makedirs(path, exist_ok=True)
    print("Downloading dataset...")
    urllib.request.urlretrieve(file_url, file_path)

with open(file_path, "r", encoding="utf-8") as f:
    text = f.read()


In [6]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [
    stoi[c] for c in s
]  # encoder: take a string, output a list of integers
decode = lambda l: "".join(
    [itos[i] for i in l]
)  # decoder: take a list of integers, output a string


In [7]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


In [8]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


In [9]:
def calc_loss(logits: torch.Tensor, targets: torch.Tensor):
    B, T, C = logits.shape
    logits = logits.view(B * T, C)
    targets = targets.view(B * T)
    loss = F.cross_entropy(logits, targets)
    return loss


In [10]:
@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits = model(X)
            loss = calc_loss(logits, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [11]:
class FeedFoward(nn.Module):
    """a simple linear layer followed by a non-linearity"""

    def __init__(self, n_in, n_out, factor, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_in, factor * n_in),
            nn.ReLU(),
            nn.Linear(factor * n_in, n_out),
            nn.Dropout(dropout),
        )

    def forward(self, x: torch.Tensor):
        return self.net(x)


In [12]:
class CombinatorialHead(nn.Module):
    def __init__(
        self,
        block_size,
        n_embd,
        head_size,
        n_value_functions,
        n_key_functions,
        n_query_functions,
        dropout,
        functions_expansion_factor,
    ):
        super().__init__()
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        self.index_input_size = n_key_functions + n_query_functions + n_value_functions
        self.keys = nn.ModuleList(
            [
                FeedFoward(
                    n_embd + self.index_input_size, head_size, functions_expansion_factor, dropout=dropout
                )
                for _ in range(n_key_functions)
            ]
        )
        self.queries = nn.ModuleList(
            [
                FeedFoward(
                    n_embd + self.index_input_size, head_size, functions_expansion_factor, dropout=dropout
                )
                for _ in range(n_query_functions)
            ]
        )
        self.values = nn.ModuleList(
            [
                FeedFoward(
                    n_embd + self.index_input_size, head_size, functions_expansion_factor, dropout=dropout
                )
                for _ in range(n_value_functions)
            ]
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B, T, C = x.shape
        outputs = []
        for ik, key in enumerate(self.keys):
            key_index = F.one_hot(torch.tensor(ik), num_classes=n_key_functions).to(x.device, x.dtype, non_blocking=True)
            for iq, query in enumerate(self.queries):
                query_index = F.one_hot(torch.tensor(iq), num_classes=n_query_functions).to(
                    x.device, x.dtype, non_blocking=True
                )
                for iv, value in enumerate(self.values):
                    value_index = F.one_hot(torch.tensor(iv), num_classes=n_value_functions).to(
                        x.device, x.dtype
                    )
                    index = torch.cat(
                        [key_index, query_index, value_index], dim=-1
                    )
                    index *= x.max()
                    index_input = index.expand(B, T, self.index_input_size)
                    function_input = torch.cat([x, index_input], dim=-1)
                    k = key(function_input)  # (B,T,hs)
                    q = query(function_input)  # (B,T,hs)
                    # compute attention scores ("affinities")
                    wei = (
                        q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
                    )  # (B, T, hs) @ (B, hs, T) -> (B, T, T)
                    wei = wei.masked_fill(
                        self.tril[:T, :T] == 0, float("-inf")
                    )  # (B, T, T)
                    wei = F.softmax(wei, dim=-1)  # (B, T, T)
                    wei = self.dropout(wei)
                    # perform the weighted aggregation of the values
                    v = value(function_input)  # (B,T,hs)
                    out = wei @ v  # (B, T, T) @ (B, T, hs) -> (B, T, hs)
                    outputs.append(out)
        return torch.cat(outputs, dim=-1)


In [13]:
class CombinatorialHeadAttention(nn.Module):
    def __init__(
        self,
        block_size,
        n_embd,
        head_size,
        n_value_functions,
        n_key_functions,
        n_query_functions,
        dropout,
        functions_expansion_factor,
        projection_expansion_factor,
    ):
        super().__init__()
        self.head = CombinatorialHead(
            block_size,
            n_embd,
            head_size,
            n_value_functions,
            n_key_functions,
            n_query_functions,
            dropout,
            functions_expansion_factor,
        )
        heads_output_size = (
            head_size * n_value_functions * n_key_functions * n_query_functions
        )
        self.proj = FeedFoward(
            heads_output_size, n_embd, projection_expansion_factor, dropout=dropout
        )

    def forward(self, x: torch.Tensor):
        out = self.head(x)
        out = self.proj(out)
        return out


In [14]:
class Block(nn.Module):
    """Transformer block: communication followed by computation"""

    def __init__(
        self,
        block_size,
        n_embd,
        head_size,
        n_value_functions,
        n_key_functions,
        n_query_functions,
        dropout,
        functions_expansion_factor,
        projection_expansion_factor,
        ffwd_expansion_factor,
        enable_ffwd_layer,
    ):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        self.sa = CombinatorialHeadAttention(
            block_size,
            n_embd,
            head_size,
            n_value_functions,
            n_key_functions,
            n_query_functions,
            dropout,
            functions_expansion_factor,
            projection_expansion_factor,
        )
        self.ln1 = nn.LayerNorm(n_embd)
        self.enable_ffwd_layer = enable_ffwd_layer
        if enable_ffwd_layer:
            self.ffwd = FeedFoward(
                n_embd, n_embd, ffwd_expansion_factor, dropout=dropout
            )
            self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x: torch.Tensor):
        x = x + self.sa(self.ln1(x))
        if self.enable_ffwd_layer:
            x = x + self.ffwd(self.ln2(x))
        return x


In [15]:
class CombinatorialTransformerModel(nn.Module):
    def __init__(
        self,
        vocab_size,
        block_size,
        n_layer,
        n_embd,
        head_size,
        n_value_functions,
        n_key_functions,
        n_query_functions,
        dropout,
        functions_expansion_factor,
        projection_expansion_factor,
        ffwd_expansion_factor,
        enable_ffwd_layer,
    ):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            *[
                Block(
                    block_size,
                    n_embd,
                    head_size,
                    n_value_functions,
                    n_key_functions,
                    n_query_functions,
                    dropout,
                    functions_expansion_factor,
                    projection_expansion_factor,
                    ffwd_expansion_factor,
                    enable_ffwd_layer,
                )
                for _ in range(n_layer)
            ]
        )
        self.ln_f = nn.LayerNorm(n_embd)  # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx: torch.Tensor):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T,C)
        x = tok_emb + pos_emb  # (B,T,C)
        x = self.blocks(x)  # (B,T,C)
        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)
        return logits

    def generate(self, idx: torch.Tensor, max_new_tokens: int):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx


In [16]:
model = CombinatorialTransformerModel(
    vocab_size,
    block_size,
    n_layer,
    n_embd,
    head_size,
    n_value_functions,
    n_key_functions,
    n_query_functions,
    dropout,
    functions_expansion_factor,
    projection_expansion_factor,
    ffwd_expansion_factor,
    enable_ffwd_layer,
)
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters()) / 1e6, "M parameters")


13.238465 M parameters


In [17]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


In [18]:
model.train()

for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(model)
        print(
            f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}"
        )

    # sample a batch of data
    xb, yb = get_batch("train")

    logits = model(xb)
    loss = calc_loss(logits, yb)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 4.2002, val loss 4.1941
step 500: train loss 2.3360, val loss 2.3551
step 1000: train loss 1.9757, val loss 2.1059
step 1500: train loss 1.7565, val loss 1.9272
step 2000: train loss 1.6362, val loss 1.8256
step 2500: train loss 1.5402, val loss 1.7510
step 3000: train loss 1.4840, val loss 1.7129
step 3500: train loss 1.4396, val loss 1.6811
step 4000: train loss 1.4074, val loss 1.6585
step 4500: train loss 1.3803, val loss 1.6380
step 5000: train loss 1.3570, val loss 1.6391
step 5500: train loss 1.3266, val loss 1.6186
step 6000: train loss 1.3156, val loss 1.6207
step 6500: train loss 1.2871, val loss 1.6032
step 7000: train loss 1.2716, val loss 1.5924
step 7500: train loss 1.2456, val loss 1.5931
step 8000: train loss 1.2331, val loss 1.5852
step 8500: train loss 1.2189, val loss 1.6065
step 9000: train loss 1.1990, val loss 1.6032
step 9500: train loss 1.1862, val loss 1.6015
step 9999: train loss 1.1661, val loss 1.5920


In [19]:
model.eval()
with torch.no_grad():
    # generate from the model
    context = torch.zeros((1, 1), dtype=torch.long, device=device)
    print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
    # open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))



I loved thee qualify foul time of thy father
Shall be thy spiritian ciltious first death.

EXTON:
IGood hom mine to those smelves touch and me a moinoce.

HENRY
BOLIO:
No, but not dime, faith: daoth lie fortune as I have,
yet so molehild, I know you have leavier, y and you tire:
God go, but mark with my ansom, take me breaks.

MERCUTIO:
Signarate on you, sir?

TYARK:
Most pay you be, sirr! is a powmascalim agised that;
why that hath dos make yours; and I have tere of her
thee absides, and the ho
